# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

Using TensorFlow backend.


# Define different types of Keras models

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

lambda_regularization = 0.20

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(units=1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    #Clear the previous tensorflow graph
    K.clear_session()

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN_6(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [6]:
num_features = len(selected_features)

#shape = len(selected_features*window_size)  
#model = get_compiled_model(shape, model_type='ann')
#tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

shape = (window_size, num_features)
model = get_compiled_model(shape, model_type='lstm')
tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)
#tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = min_max_scaler

# Load Data

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 125

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[125. 125. 125. 125. 125.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
   0.07882403]
 [-0

In [7]:
#Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 125

tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 30, 14)
(17731,)
Testing data (X, y)
(100, 30, 14)
(100,)
Printing first 5 elements

Training data (X, y)
[[[-0.63253012 -0.18639634 -0.38048616 ... -0.33333333  0.42635659
    0.44932339]
  [-0.43373494 -0.09396119 -0.29473329 ... -0.33333333  0.33333333
    0.46202706]
  [-0.31325301 -0.26095487 -0.25894666 ... -0.66666667  0.25581395
    0.24275062]
  ...
  [-0.31325301 -0.48550251 -0.43011479 ... -0.66666667  0.34883721
    0.07677437]
  [-0.57831325 -0.39873556 -0.36731938 ... -0.16666667  0.2248062
    0.28555648]
  [-0.40361446 -0.01983867 -0.53308575 ... -0.66666667  0.41085271
    0.42723005]]

 [[-0.43373494 -0.09396119 -0.29473329 ... -0.33333333  0.33333333
    0.46202706]
  [-0.31325301 -0.26095487 -0.25894666 ... -0.66666667  0.25581395
    0.24275062]
  [-0.31325301 -0.48768

# Train the model and test some Tunable Model functionalities

In [10]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#shape = len(selected_features*window_size)  
#model = get_compiled_model(shape, model_type='ann')
#tModel.change_model('ModelRUL_SN_1', model, 'keras')

shape = (window_size, num_features)
model = get_compiled_model(shape, model_type='lstm')
tModel.change_model('ModelRUL_RNN_1', model, 'keras')

#tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = min_max_scaler

tModel.epochs = 100
tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 2s 100us/step - loss: 8134.2347 - mean_squared_error: 8131.5858
Epoch 2/100
17731/17731 [==============================] - 1s 73us/step - loss: 7614.1470 - mean_squared_error: 7611.3031
Epoch 3/100
17731/17731 [==============================] - 1s 72us/step - loss: 6885.1259 - mean_squared_error: 6881.8874
Epoch 4/100
17731/17731 [==============================] - 1s 72us/step - loss: 6273.1978 - mean_squared_error: 6269.4876
Epoch 5/100
17731/17731 [==============================] - 1s 72us/step - loss: 5719.2102 - mean_squared_error: 5714.9761
Epoch 6/100
17731/17731 [==============================] - 1s 72us/step - loss: 5169.6704 - mean_squared_error: 5164.8595
Epoch 7/100
17731/17731 [==============================] - 1s 72us/step - loss: 4624.5877 - mean_squared_error: 4619.1481
Epoch 8/100
17731/17731 [==============================] - 1s 72us/step - loss: 4093.7990 - mean_squared_error: 4087.6861
Epoch 9/100
17731/17731

17731/17731 [==============================] - 1s 73us/step - loss: 166.6383 - mean_squared_error: 147.6006
Epoch 69/100
17731/17731 [==============================] - 1s 74us/step - loss: 166.2132 - mean_squared_error: 147.1852
Epoch 70/100
17731/17731 [==============================] - 1s 73us/step - loss: 164.8286 - mean_squared_error: 145.8057
Epoch 71/100
17731/17731 [==============================] - 1s 72us/step - loss: 163.7993 - mean_squared_error: 144.7866
Epoch 72/100
17731/17731 [==============================] - 1s 72us/step - loss: 163.2183 - mean_squared_error: 144.2196
Epoch 73/100
17731/17731 [==============================] - 1s 73us/step - loss: 164.4057 - mean_squared_error: 145.4189
Epoch 74/100
17731/17731 [==============================] - 1s 73us/step - loss: 163.9166 - mean_squared_error: 144.9425
Epoch 75/100
17731/17731 [==============================] - 1s 72us/step - loss: 167.9709 - mean_squared_error: 149.0135
Epoch 76/100
17731/17731 [===================

In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 1ms/step
scores
Engine 1, Predicted RUL [114.50039], Rounded RUL 114.0, Real RUL 112.0
Engine 2, Predicted RUL [120.39676], Rounded RUL 120.0, Real RUL 98.0
Engine 3, Predicted RUL [46.24574], Rounded RUL 46.0, Real RUL 69.0
Engine 4, Predicted RUL [87.45438], Rounded RUL 87.0, Real RUL 82.0
Engine 5, Predicted RUL [113.72997], Rounded RUL 113.0, Real RUL 91.0
Engine 6, Predicted RUL [108.81033], Rounded RUL 108.0, Real RUL 93.0
Engine 7, Predicted RUL [106.30003], Rounded RUL 106.0, Real RUL 91.0
Engine 8, Predicted RUL [117.51744], Rounded RUL 117.0, Real RUL 95.0
Engine 9, Predicted RUL [122.51858], Rounded RUL 122.0, Real RUL 111.0
Engine 10, Predicted RUL [85.96573], Rounded RUL 85.0, Real RUL 96.0
Engine 11, Predicted RUL [91.19946], Rounded RUL 91.0, Real RUL 97.0
Engine 12, Predicted RUL [107.63626], Rounded RUL 107.0, Real RUL 124.0
Engine 13, Predicted RUL [79.05565], Rounded RUL 79.0, Real RUL 95.0
Engine 14, Predicted RUL [116.9